In [52]:
import pandas as pd
import os

In [49]:
def json_to_df(filepath, filename):
    try:
        df = pd.read_json(filepath + filename)
        return df
    except:
        print("Invalid file") 

def general_dataset_info(dataset):
    print("----------Null Values----------\n", dataset.isnull().sum())
    print("\n----------Duplicate Values----------\n", dataset.duplicated().sum())
    print("\nTotal entries:", len(dataset))

def join_players_and_team(filepath):
    teams = json_to_df(filepath, "teams.json")
    teams = teams[['id', 'name', 'acronym', 'home_league_id']]

    aggregated_teams = teams.groupby('id').agg({
    'name': lambda x: ', '.join(x.unique()),   # Combine names into a comma-separated string
    'acronym': lambda x: ', '.join(x.unique()), # Combine acronyms into a comma-separated string
    'home_league_id': 'first'
}).reset_index()
    
    players = json_to_df(filepath, "players.json")
    players = players[['handle', 'first_name', 'last_name', 'home_team_id']]
    players = players.drop_duplicates()
    
    leagues = json_to_df(filepath, "leagues.json")
    leagues = leagues[['league_id', 'region', 'name']]


    merged_df = pd.merge(players, aggregated_teams, left_on='home_team_id', right_on='id', how='left')
    merged_df = pd.merge(merged_df, leagues, left_on='home_league_id', right_on='league_id', how='left')
    merged_df.rename(columns={
    'name_x': 'team_name',
    'name_y': 'league_name',

    }, inplace=True)

    merged_df = merged_df[['handle', 'first_name', 'last_name','team_name', 'acronym', 'region', 'league_name']]
    
    return merged_df


In [26]:
STATS = 'Z:\\'
CH_FOLDER = 'Z:\\vct-challengers\esports-data\\'
GC_FOLDER = 'Z:\\game-changers\esports-data\\'
INTL_FOLDER = 'Z:\\vct-international\esports-data\\'
PICKRATES = 'Z:\\'

In [4]:
stats_df = json_to_df(STATS, "stats_data.json")

In [46]:
pickrates_df = json_to_df(PICKRATES, "map_pickrates.json")

Make dataframe for each division with players, their team, and their league.

In [5]:
ch_df = join_players_and_team(CH_FOLDER)
gc_df = join_players_and_team(GC_FOLDER)
intl_df = join_players_and_team(INTL_FOLDER)

ch_df = ch_df.drop_duplicates()
gc_df = gc_df.drop_duplicates()
intl_df = intl_df.drop_duplicates()

print("CHALLENGERS DATA\n")
general_dataset_info(ch_df)

print("\nGAME CHANGERS DATA\n")
general_dataset_info(gc_df)

print("\nINTERNATIONAL DATA\n")
general_dataset_info(intl_df)

CHALLENGERS DATA

----------Null Values----------
 handle          0
first_name     33
last_name       6
team_name       0
acronym         0
region          0
league_name     0
dtype: int64

----------Duplicate Values----------
 0

Total entries: 5321
----------Null Values----------
 handle         0
first_name     0
last_name      0
team_name      7
acronym        7
region         7
league_name    7
dtype: int64

----------Duplicate Values----------
 0

Total entries: 2617

INTERNATIONAL DATA

----------Null Values----------
 handle         0
first_name     0
last_name      0
team_name      0
acronym        0
region         0
league_name    0
dtype: int64

----------Duplicate Values----------
 0

Total entries: 1542


PROBABLY DONT JOIN THEM ALL TOGETHER SINCE SOME PLAYERS PLAY IN MULTIPLE LEAGUES SO THEN THERE ARE DUPLICATES AND IT BECOMES REALLLLLY ANNOYING

In [15]:
# combined_df = pd.concat([ch_df, gc_df, intl_df], ignore_index=True)
# combined_df.reset_index(drop=True, inplace=True)
# general_dataset_info(combined_df)
# stats_df.shape[0]

In [16]:
# duplicates = combined_df[combined_df[['handle', 'first_name', 'last_name']].duplicated(keep=False)]

# # Group by the column with duplicates and compare other columns
# def compare_columns(group):
#     differences = (group.iloc[1:] != group.iloc[0]).any()
#     return differences[differences]

# duplicate_groups = duplicates.groupby('handle').apply(compare_columns)
# print(duplicate_groups)

Add role column for each player in the stats dataframe

In [22]:
DUELIST = 'duelist'
CONTROLLER = 'controller'
INITIATOR = 'initiator'
SENTINEL = 'sentinel'

agent_to_role = {
    'yoru': DUELIST,
    'omen': CONTROLLER,
    'astra': CONTROLLER,
    'brimstone': CONTROLLER,
    'jett': DUELIST,
    'phoenix': DUELIST,
    'sage' : SENTINEL,
    'sova' : INITIATOR,
    'viper' : CONTROLLER,
    'cypher' : SENTINEL,
    'reyna' : DUELIST,
    'killjoy' : SENTINEL,
    'breach' : INITIATOR,
    'raze' : DUELIST,
    'skye' : INITIATOR,
    'kay/o': INITIATOR,
    'chamber': SENTINEL,
    'neon': DUELIST,
    'fade' : INITIATOR,
    'harbor': CONTROLLER,
    'gekko' : INITIATOR,
    'deadlock' : CONTROLLER,
    'iso' : DUELIST,
    'clove' : CONTROLLER,
    'vyse' : SENTINEL
}

def get_roles(agent_list):
    roles = {agent_to_role[agent] for agent in agent_list if agent in agent_to_role}
    return list(roles)

stats_df['Roles'] = stats_df['Agents'].apply(get_roles)

stats_df.head()

,Rnd,R2.0,ACS,K:D,KAST,ADR,KPR,APR,FKPR,FDPR,...,CL,KMax,K,D,A,FK,FD,Player,Agents,Roles
0,340,1.37,283.9,1.44,76%,188.5,1.00,0.21,0.16,0.10,...,5/39,33,341,237,70,54,35,florescent,"[jett, raze]",[duelist]
1,327,1.36,308.4,1.45,75%,187.3,1.05,0.26,0.31,0.21,...,5/14,32,343,237,84,101,69,Miku,"[yoru, raze, jett]",[duelist]
2,423,1.34,297.2,1.49,73%,198.6,1.05,0.22,0.26,0.18,...,10/32,30,444,297,91,108,75,Lied,"[raze, iso, gekko]","[duelist, initiator]"
3,328,1.31,287.9,1.41,74%,182.8,1.03,0.17,0.26,0.17,...,4/17,30,339,241,55,86,57,miNt,"[raze, jett]",[duelist]
4,349,1.30,227.8,1.49,76%,148.7,0.85,0.22,0.07,0.05,...,10/55,25,297,199,77,25,17,mmonch,"[cypher, killjoy, deadlock]","[sentinel, controller]"


In [74]:
pickrates_df['Map'] = pickrates_df['Map'].apply(lambda x: 'All' if x.strip() == '' else x.split('\\')[-1].strip())
pickrates_df.rename(columns={'#': 'Games Played'}, inplace=True)
print(pickrates_df)

      Map  Games Played ATK WIN DEF WIN  omen  sova viper cypher raze fade  \
0     All            86     52%     48%   62%   52%   50%    42%  41%  34%   
1   Lotus            20     55%     45%  100%    0%   68%    18%  75%  90%   
2    Bind            15     54%     46%   13%    0%   90%    17%  87%  63%   
3  Sunset            14     53%     47%   93%   75%    0%    93%  46%  11%   
4   Abyss            12     51%     49%   50%   96%   21%    75%   0%   4%   
5  Icebox            12     52%     48%    0%   92%  100%     0%   0%   0%   
6   Haven             9     48%     52%  100%   89%   17%    78%   6%   0%   
7  Ascent             4     47%     53%   88%  100%    0%    38%   0%   0%   

   ... skye harbor deadlock sage clove chamber  iso phoenix reyna vyse  
0  ...   8%     7%       5%   4%    4%      3%   1%      1%    0%   0%  
1  ...   3%     0%       3%   0%   15%      8%   0%      0%    0%   0%  
2  ...  37%     0%      10%   0%    0%      7%   0%      0%    0%   0%  
3  ..

In [75]:
NEW_FILEPATH = "Z:\\Cleaned Jsons\\"

os.makedirs(NEW_FILEPATH, exist_ok=True)

ch_df.to_json(NEW_FILEPATH+'ch_players.json')
gc_df.to_json(NEW_FILEPATH+'gc_players.json')
intl_df.to_json(NEW_FILEPATH+'intl_players.json')
stats_df.to_json(NEW_FILEPATH+'allplayer_stats.json')
pickrates_df.to_json(NEW_FILEPATH+'agent_pickrates.json')